#  GTA (1850-2015) and AMO (1857-2015)

For filtering (Kalman filter and fixed interval smoothing) I will use toolbok [E4](https://www.ucm.es/e-4/):

In [7]:
clear
e4init


            XXXXXX 4 4      XXXXX  XX     XX  XX  XXXXXX (c)
           XX      444     XX  XX XX     XX  XX  XX
          XXXX       4    XXXXX  XX     XX  XX  XXXXXX
         XX              XX     XX     XX  XX      XX
        XXXXXX          XX     XXXXXX XXXXXX  XXXXXX

    Toolbox for State Space Estimation of Econometric Models
 
*********************** Options set by user ********************
Filter. . . . . . . . . . . . . : KALMAN
Scaled B and M matrices . . . . : NO
Initial state vector. . . . . . : AUTOMATIC SELECTION
Initial covariance of state v.  : IDEJONG
Variance or Cholesky factor?  . : VARIANCE
Optimization algorithm. . . . . : BFGS
Maximum step length . . . . . . : 0.100000
Stop tolerance. . . . . . . . . : 0.000010
Max. number of iterations . . . :       75
Verbose iterations. . . . . . . : YES
****************************************************************
 
 


It is also necessary to load the control toolbox

In [8]:
pkg load control

We load `gta.data` that I have download from [Global and Hemispheric Temperature Anomalies - Land and Marine Instrumental Records](https://cdiac.ess-dive.lbl.gov/trends/temp/jonescru/jones.html) 

In [9]:
load gta.data

The fourteenth column of `gta` matrix contains the annual data for the years 1850--2015. For convenience I define the variable `y` with the annual data (i.e. periodicity one: `p=1`).

In [10]:
y = gta(:,14);
p = 1;

Professor Peter C. Young has kindly sent me the following data sets. 

In [11]:
load data4Marcos.mat % GTA and TRF (1857-2015)

In [12]:
load amo4Marcos.mat  % AMO (1857-2015)

In [13]:
%%plot gnuplot

In [14]:
plot([Z,amon])

... better if the figures are inserted in the notebook itself in 'png' format...

In [15]:
%%plot --format png

I'll try to improve the graph

In [16]:
#y=Z(:,1);
dates = gta(:,1);    # first column of gta contains the years
yr = datenum (dates,1,1);
plot(yr, y, 'r')
set (gca(), "xtick", datenum (1850:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
ylim([-1.3, 1])
title('Global Annual Temperature Anomalies (degrees C) 1850-2015')
box off;

## A DHR model for GTA series (first attempt)

I will define a range of AR orders to work with. Since the data are annual, it is better to use long autoregressive orders to avoid identifying noisy trends (I want the pseudo-spectrum of the trend component to be close to a spectral peak very close to the vertical axis, and this is only possible if we use a high order autoregressive polynomial).

In [17]:
rgar = [51:70];

If we do not indicate an a priori model, and also the periodicity of the data is annual `(p=1)`, then the `LDHR` toolbox will try to fit (_whenever possible_) an IRW trend plus an irregular component model.

In [18]:
format long
[VAR0,P0,TVP0,oar0]=autodhr(y,p,[],rgar)
NVR = VAR0(2)/VAR0(1)

    dhr2rf at line 36 column 35
    autodhr at line 342 column 31

    dhr2rf at line 36 column 35
    autodhr at line 342 column 31

    thd2ss at line 36 column 36
    thd2ssc at line 69 column 48
    thd2ss at line 56 column 36
    dhr2rf at line 36 column 35
    autodhr at line 342 column 31

    thd2ss at line 36 column 36
    thd2ssc at line 69 column 48
    thd2ss at line 56 column 36
    dhr2rf at line 36 column 35
    autodhr at line 342 column 31

    dhr2rf at line 38 column 41
    autodhr at line 342 column 31

    dhr2rf at line 38 column 41
    autodhr at line 342 column 31

VAR0 =

   4.658045203166136e-03   2.146566701538700e-06

P0 = Inf
TVP0 =

   1
   1

oar0 = 63
NVR = 4.608299421567762e-04


`TVP` indicates two unit roots for a component with periodicity `P=Inf`, that is, an IRW trend. VAR contains the estimated variances, `oar` is the order of the AR polinomial used to identify and adjust the model.

Using this model, we filter the series (by Fixed-Interval Smoothing) to obtain the components

In [19]:
[Trend0,season0,cycle0,irreg0]=dhrfilt(y,P0,TVP0,VAR0,p,0,0);

    e4trend at line 61 column 18
    dhrfilt at line 127 column 56

    e4trend at line 61 column 18
    dhrfilt at line 127 column 56

    thd2ss at line 46 column 36
    fismod at line 36 column 33
    e4trend at line 103 column 22
    dhrfilt at line 127 column 56



Let's draw them (to make the figure clearer, I will move down the irregular component).

In [20]:
plot(yr, Trend0, '-b', "linewidth", 4)
set (gca(), "xtick", datenum (1850:15:2015,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yr, y,'-r')
plot(yr, irreg0-1)
plot(yr, -ones(length(y)), ':k')
ylim([-1.3, 1])
title('Estimated trend and irregular component (shifted -1 units)')
box off;

The trend shows a cyclical behaviour. We can fit an AR polynomial and observe the periodicity associated with the roots of higher modulus and lower frequencies; we then find a cycle of about 63 years.

In [21]:
[ARSPT0, S0, LAGS0, AR0, ROOTS0, NORM0, P0] = aresp(Trend0,rgar,0);

I will show the periodicities of the roots associated to lower frequencies and norms greater than 0.985

In [22]:
[s,i]=sort(P0,'descend');
format bank
n=length(P0);
R=[[1:n]',[P0,NORM0](i(1:n),:)];
R(R(:,3)>.985,:)

ans =

     1.00      Inf     1.04
     2.00   327.35     0.99
     3.00    63.62     1.00
     4.00    21.52     0.99
     8.00     9.11     0.99
    12.00     5.25     0.99
    15.00     4.14     0.99
    17.00     3.55     0.99
    20.00     2.88     0.99
    23.00     2.49     1.00
    26.00     2.15     0.99
    27.00     2.04     1.00



In [23]:
#63./[1:31] % all harmonics
#lcm(21*4, 9*4, 5.25*4)/4
#63./[3,7,12]

[arespt,w]=esptarma(1,AR0);
per=63./[0,1,3,7,12,15,18,22,25,31];   % selected harmonics
semilogy(w,arespt);
set(gca,'XTick',2*pi./per)
set(gca,'XTickLabel',{'Inf','63','21','9','5.25','4.2','3.5','2.86','2.52','2.03'})
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
title(sprintf('f(w): Ar-Espectro de la Tendencia inicial de GTA. AR(%g)',LAGS0))
xlabel('period (in years)')
ylabel('f(w) (Escala en logs)')

From this exploration of the data, it appears that there is a 63 year cycle and several harmonics

Hence, `PaP` will be the "_a priori_" periodicity of our DHR components model

In [24]:
format bank
PaP = 63./[0,1,3,7,12,15,18,22,25,31] 

PaP =

     Inf   63.00   21.00    9.00    5.25    4.20    3.50    2.86    2.52    2.03



Let's try to identify and estimate the parameters of a DHR model 

I will only provide "_a priori_" periodicities `PaP`, the range `rgar` of AR polinomials to explore, the periodicity of the data, i.e. `p=1` (anual data), and the data itself `y`.

In [25]:
format long
[VAR,P,TVP,oar]=autodhr(y,1,[],rgar,PaP)

VAR =

 Columns 1 through 3:

   1.269316838490171e-03   2.504158752336266e-07   4.789119643723519e-06

 Columns 4 through 6:

   7.829299091809946e-06   1.212707750623201e-05   4.955304465038746e-06

 Columns 7 through 9:

   2.158923818838853e-05   2.397675184599739e-06   4.108799268284472e-06

 Columns 10 and 11:

   4.270389081699910e-06   3.893251747927233e-06

P =

 Columns 1 through 3:

                     Inf   6.300000000000000e+01   2.100000000000000e+01

 Columns 4 through 6:

   9.000000000000000e+00   5.250000000000000e+00   4.200000000000000e+00

 Columns 7 through 9:

   3.500000000000000e+00   2.863636363636364e+00   2.520000000000000e+00

 Column 10:

   2.032258064516129e+00

TVP =

   1   1   1   1   1   1   1   1   1   1
   1   0   0   0   0   0   0   0   0   0

oar = 60


It seems to work! Using the roots of an AR polynomial of order 60, it identifies an IRW trend and three RW cyclic components.

Let's obtain the components by filtering the series:

In [26]:
[Trend,season,cycle,irreg]=dhrfilt(y,P,TVP,VAR,p,2,0);

In [27]:
plot(Trend)

In [28]:
plot([cycle(:,1),cycle(:,2)]) % full cycle and and the harmonic of period 63

As can be seen in the following figure, the Trend-Cycle fits the evolution of the series quite well

In [29]:
plot(yr, Trend(:,1)+cycle(:,1), '-k', "linewidth", 4)
set (gca(), "xtick", datenum (1850:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yr, y,'-r')
plot(yr, irreg-1)
plot(yr, -ones(length(y)), ':k')
ylim([-1.3, 1])
title('Estimated Trend-Cycle and irregular component (shifted -1 units)')
box off;

Although I think it is much more illustrative to plot the long-term Trend on the one hand, and the Cycle on the other.

it seems that the _Cycle_ apparently replicates the behaviour of the [Atlantic Multidecadal Oscillation (AMO)](https://www.psl.noaa.gov/data/timeseries/AMO/) (but I need AMO data to check).

In [30]:
plot(yr, Trend(:,1), '-k', "linewidth", 4)
set (gca(), "xtick", datenum (1850:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yr, y,'-r')
plot(yr, irreg-1)
plot(yr, -ones(length(y)), ':k')
plot(yr, cycle(:,1)+0.7, '-b')
plot(yr, cycle(:,2)+0.7,'--k')
plot(yr, ones(length(y))*.7, ':k')
ylim([-1.3, 1])
title('Estimated Trend, Cycle (shifted +0.7 units) and irregular component (shifted -1 units)')
box off;

## A DHR model for AMO series (first attempt)

Let's try the same with AMO series (Atlantic Multidecadal Oscillation)

In [31]:
yramon = datenum (dates(7:end),1,1);
plot(yramon, amon(:,1), '-r', "linewidth", 1)
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
ylim([-1.3, 1])
title('Atlantic Multidecadal Oscillation 1857-2015')
box off;

In [32]:
[VARamo0,Pamo0,TVPamo0,oaramo0]=autodhr(amon,p,[],rgar)

This AMO series is linearly detrended data. Nevertheless, DHR identify just only a trend component. When we see it it clear that this IRW trend captures a smooth cycle.

In [33]:
[TrendAMO0,seasonAMO0,cycleAMO0,irregAMO0]=dhrfilt(amon,Pamo0,TVPamo0,VARamo0,p,2);

error: 'Pamo0' undefined near line 1, column 1


In [34]:
yramon = datenum (dates(7:end),1,1);
plot(yramon, TrendAMO0(:,1)+cycleAMO0(:,1), '-b', "linewidth", 4)
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, amon,'-r')
plot(yramon, irregAMO0-1)
plot(yramon, -ones(length(amon)), ':k')
ylim([-1.3, 1])
title('Estimated trend and irregular component (shifted -1 units)')
box off;

error: 'TrendAMO0' undefined near line 1, column 1
error: 'irregAMO0' undefined near line 1, column 1


# There is something suspicious

It resembles the 63-years cycle GTA component, but is not as regular as the GTA cycle. Mainly at the first part of the sample. Why? I guess both the "Global Temperature" and the Atlantic Ocean must be warmed by the sun and both must be affected by "Radiative Forcing Inputs". So why does the AMO show a different oscillation pattern?

In [35]:
plot(TrendAMO0)

error: 'TrendAMO0' undefined near line 1, column 1


The AMO data has been linearly de-trended, but we have seen that the GTA has a non-linear trend. If we subtract a linear trend from the very first GTA trend, what does it look like?

In [36]:
plot(detrend(Trend0(7:end),1))

__My guess:__ the cycles of GTA and AMO looks like no so similar due to the "de-trending procedure" of AMO data

In fact, we can find in the AMO series some harmonics of the 63-year cycle (in particular those associated with the 21- and 9-year cycles).

In [37]:
[ARSPT, S, LAGS, AR, ROOTS, NORM, P] = aresp(amon,[58:70],0,0);

[s,i]=sort(P,'descend');
format bank
n=length(P);
R=[[1:n]',[P,NORM](i(1:n),:)];
R(R(:,3)>.98,:)

ans =

    1.00     Inf    0.99
    2.00   67.08    1.00
    3.00   30.66    0.98
    4.00   21.08    0.98
    8.00    9.00    1.00
    9.00    7.37    1.00
   11.00    5.99    0.99
   12.00    5.38    0.98
   13.00    5.07    0.98
   16.00    4.05    0.99
   17.00    3.77    0.99
   18.00    3.55    1.00
   19.00    3.32    0.98
   20.00    3.18    0.98
   22.00    2.86    1.00
   24.00    2.65    0.98
   25.00    2.49    0.99
   27.00    2.29    0.99
   28.00    2.20    0.99
   30.00    2.09    0.99
   31.00    2.00    1.00



In [38]:
[arespt,w]=esptarma(1,AR);
per=63./[0,1,3,7,12,15,18,22,25,31]; % harmonics selected from the analysis of the GTA series
semilogy(w,arespt);
set(gca,'XTick',2*pi./per)
set(gca,'XTickLabel',{'Inf','63','21','9','5.25','4.2','3.5','2.86','2.52','2.03'})
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
title(sprintf('f(w): Ar-Espectro de la Tendencia inicial de AMO. AR(%g)',LAGS))
xlabel('period (in years)')
ylabel('f(w) (Escala en logs)')

Therefore, if my guess is correct, although the 63-year cycle of the AMO series is very much affected by the de-trending process, the other harmonics should be there. 

So, let's try with a DHR model for AMO that leaves the cycles longer that 21 year in the trend

In [39]:
format bank
PaP = 63./[0,3,7,12,15,18,22,25,31]    # without 63 cycle for AMO

PaP =

     Inf   21.00    9.00    5.25    4.20    3.50    2.86    2.52    2.03



In [40]:
format long
[VAR,P,TVP,oar]=autodhr(amon,p,[],rgar,PaP,[1,1;1,0],1) % I will force the model

VAR =

 Columns 1 through 3:

   1.040512707088203e-02   1.029261361500811e-05   2.891383552680393e-05

 Columns 4 through 6:

   1.306375565383388e-05   2.657449708426246e-05   1.147135085989245e-05

 Columns 7 through 9:

   1.584639953027698e-05   2.260898820176219e-05   2.492320084617676e-05

 Column 10:

   4.359045722603054e-05

P =

 Columns 1 through 3:

                     Inf   2.100000000000000e+01   9.000000000000000e+00

 Columns 4 through 6:

   5.250000000000000e+00   4.200000000000000e+00   3.500000000000000e+00

 Columns 7 through 9:

   2.863636363636364e+00   2.520000000000000e+00   2.032258064516129e+00

TVP =

   1   1   1   1   1   1   1   1   1
   1   0   0   0   0   0   0   0   0

oar = 58


In [41]:
[TrendAMO,seasonAMO,cycleAMO,irregAMO]=dhrfilt(amon,P,TVP,VAR,p,2,0);

In [42]:
plot(TrendAMO)

In [43]:
plot(cycleAMO(:,1))

In [44]:
plot(irregAMO)

In [45]:
yramon = datenum (dates(7:end),1,1);
plot(yramon, TrendAMO(:,1)+cycleAMO(:,1), '-k', "linewidth", 4)
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, amon,'-r')
plot(yramon, irregAMO-1)
plot(yramon, -ones(length(amon)), ':k')
ylim([-1.3, 1])
title('Estimated Trend-Cycle for AMO and irregular component (shifted -1 units)')
box off;

In [46]:
yramon = datenum (dates(7:end),1,1);
plot(yramon, TrendAMO(:,1), '-k', "linewidth", 4)
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, amon,'-r')
plot(yramon, irregAMO-1)
plot(yramon, -ones(length(amon)), ':k')
plot(yramon, cycleAMO(:,1)+0.7, '-b')
ylim([-1.3, 1])
title('Estimated Trend, cycle and irregular components for AMO series (shifted -1 units)')
box off;

# Without the 63-year harmonic, both cycles look similar.

The estimated cycles for the GTA and AMO series with 21-year oscillations or less are quite similar.

In [47]:
plot(yramon, cycleAMO(:,1), '-k')
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, sum(cycle(7:end,3:end),2),'-b')
title('Estimated cycle componentes for GTA and AMO series (only oscilations of 21 years or less)')
box off;

In [48]:
plot(yramon, irregAMO, '-k')
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, irreg(7:end,1),'-b')
title('Irregular components for GTA and AMO series')
box off;

In [49]:
TC=Trend(:,1)+cycle(:,2);
plot(yramon, TC(7:end), '-k')
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, TrendAMO,'-r')
ylim([-1.3, 1])
title('Estimated Trend-Cycle components (including oscilations of 63 years or more) for GTA and AMO')
box off;

If we linearly de-trend the trend-cycle of GTA we get the following graph

In [50]:
TC=Trend(:,1)+cycle(:,2);
plot(yramon, TrendAMO,'-r')
set (gca(), "xtick", datenum (1857:15:2022,1,1));
datetick ("x", "YYYY", "keepticks");
set (gca(), "xgrid", "on");
set(gca,'GridLineStyle','--')
hold on
plot(yramon, (detrend(TC,1))(7:end),'-b')
ylim([-1.3, 1])
title('Estimated Cycle components (with oscilations of 63 years or more)')
box off;

Therefore, perhaps the modelling of the TRF, GTA and AMO will improve and simplify by using the AMO data before the linear de-trending process (I do not know if the original AMO data are available). Otherwise, since the frequency and phase of the AMO cycles are modified,  that modelling is affected.

# The following is no correct, but it ilustrates my point

Imagine that the GTA linear trend is the same line that was subtracted from the original AMO data. Then we can add the GTA linear trend to the AMO data to recover the original Ocenan Atlantic surface temperature data (I am sure those linear trend are not the same, but imagine both are the same line).

In [51]:
plot(y)

In [52]:
z=y-detrend(y,1);
w=z(7:end)+amon;

So, let's plot GTA and AMO (with its "recovered" linear trend)  (`w` is the new AMO)

In [53]:
plot([y(7:end),w])

Now we can estimate the DHR model trying to get its 63-years cycle

In [54]:
PaP = 63./[0,1,3,7,12,15,18,22,25,31];
format long
[VARw,Pw,TVPw,oarw]=autodhr(w,p,[],rgar,PaP,[1,1;1,0],1);

In [55]:
[TrendW,seasonW,cycleW,irregW]=dhrfilt(w,Pw,TVPw,VARw,p,2,0);

In [56]:
J=2;
plot([cycle(7:end,J),cycleW(:,J)])

# My point

As you can see, the cycles of period 63 are shifted but they are very similar. What I mean is that __I am afraid that by subtracting a linear trend the authors of the data have modified the properties of the cycle.__

As you can see, the cycles of period 63 are shifted but they are very similar. What I mean is that I am afraid that by subtracting a linear trend the authors of the data have modified the properties of the cycle. 

So probably, the original AMO cycles are more similar to the GTA cycles than it seems in the published data (something that we can see with the harmonics corresponding to the cycles shorter than 63 years; since the spectral peaks for those harmonics are not so close to zero, the de-trending filter is not so harmful for them).

I don't know if I have made myself clear.

In [57]:
plot([cycle(7:end,1),cycleW(:,1)])

In [58]:
plot([Trend(7:end,1),TrendW(:,1)])

In [59]:
help detrend

'detrend' is a function from the file /usr/share/octave/6.2.0/m/signal/detrend.m

 -- detrend (X, P)
     If X is a vector, 'detrend (X, P)' removes the best fit of a
     polynomial of order P from the data X.

     If X is a matrix, 'detrend (X, P)' does the same for each column in
     X.

     The second argument P is optional.  If it is not specified, a value
     of 1 is assumed.  This corresponds to removing a linear trend.

     The order of the polynomial can also be given as a string, in which
     case P must be either "constant" (corresponds to 'P=0') or "linear"
     (corresponds to 'P=1').

     See also: polyfit.

Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW
at https://www.octave.org and via the help@octave.org
mailing list.
